In [1]:
import findspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.window import Window
import pyspark.sql.functions as sf
from pyspark.sql.functions import concat_ws
from datetime import datetime, timedelta
from pyspark.sql import functions as F
import os

import json
from openai import OpenAI

from pyspark.sql.functions import coalesce, lit, col, when, concat

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores", 8).getOrCreate()

In [3]:
from pyspark.sql.functions import col, month, to_timestamp

def ETL_all_day(path):
    folder_list = os.listdir(path)

    all_paths = [f"{path}/{folder}/*.parquet" for folder in folder_list]
    df = spark.read.parquet(*all_paths)
    df = df.withColumn("datetime_ts", to_timestamp(col("datetime")))
    df = df.withColumn("Month", month(col("datetime_ts")))
    df = df.filter((col("action") == "search") & (col("user_id").isNotNull()) & (col("keyword").isNotNull()))
    return df.cache()

def most_search(df, month):
    df = df.filter(col("Month") == month)
    df = (df.groupBy("user_id", "keyword", "month").count().withColumnRenamed("count", "Total_search"))
    window = Window.partitionBy("user_id").orderBy(col("Total_search").desc())
    df = df.withColumn("Rank", row_number().over(window))
    df = (
        df.filter(col("Rank") == 1)
          .withColumnRenamed("keyword", "Most_Search")
          .select("user_id", "Most_Search", "Month")
    )
    return df

def import_to_postgresql(result):
    url = "jdbc:postgresql://localhost:5432/test_etl"
    properties = {
        "driver": "org.postgresql.Driver",
        "user": "postgres",
        "password": "1"  
    }

    (
        result.write.format("jdbc")
        .option("url", url)
        .option("dbtable", "final_project_bigdata")
        .option("user", properties["user"])
        .option("password", properties["password"])
        .option("driver", properties["driver"])
        .mode("append")  
        .save()
    )

    print("Data import successfully!")



In [4]:
label = spark.read.csv(
    "D:/study/output/most_search_values_inner.csv",
    header=True,
    inferSchema=True
)

pdf = label.toPandas()

client = OpenAI(api_key="sk-12")

def classify_batch(movie_list):
    if not movie_list:
        return {}
    prompt = f"""
    Bạn là hệ thống phân loại phim, show truyền hình và nội dung giải trí.  

    Dữ liệu đầu vào: danh sách tên (có thể sai chính tả, viết tắt, thiếu chữ).  

    Nhiệm vụ của bạn:
    1. Nhận diện tên gần đúng nhất.  
    2. Xác định thể loại chính. Nếu thể loại chưa có trong danh sách, bạn được phép tự tạo ra một nhãn thể loại ngắn gọn, dễ hiểu.  

    Một số nhóm gợi ý:  
    - Action  
    - Romance  
    - Comedy  
    - Horror  
    - Animation  
    - Drama  
    - C Drama  
    - K Drama  
    - Sports  
    - Music  
    - Reality Show  
    - TV Channel  
    - News  
    - Other  

    Trả lời DUY NHẤT 1 JSON object hợp lệ.  
    Không thêm giải thích, không thêm chữ nào khác ngoài JSON.  

    Ví dụ:  
    {{"Titanic": "Romance", "VTV6": "TV Channel", "Tuyển Việt Nam": "Sports"}}  

    Danh sách:  
    {movie_list}
    """

    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content":prompt}],
            temperature=0
        )
        text = resp.choices[0].message.content.strip()
        # Lấy JSON
        start, end = text.find("{"), text.rfind("}")
        if start == -1 or end == -1:
            return {m: "Other" for m in movie_list}
        parsed = json.loads(text[start:end+1])
        # Map kết quả
        return {title: parsed.get(title, "Other") for title in movie_list}
    except Exception as e:
        print("Error:", e)
        return {m: "Other" for m in movie_list}


def classify_all(movies, batch_size=50):
    all_mapping = {}
    for i in range(0, len(movies), batch_size):
        batch = movies[i:i+batch_size]
        print(f"xử lý batch {i//batch_size+1} ({len(batch)} items)...")
        batch_mapping = classify_batch(batch)
        all_mapping.update(batch_mapping)
    return all_mapping


# Tìm Most_Search
col_candidates = [c for c in pdf.columns if "Most_Search" in c]
if not col_candidates:
    raise ValueError("Không tìm thấy cột chứa 'Most_Search'")
col_name = col_candidates[0]
# Lấy danh sách movies
movies = pdf[col_name].dropna().astype(str).tolist()
# Phân loại theo batch
mapping = classify_all(movies, batch_size=50)
# Thêm cột cateogry
pdf["Category"] = pdf[col_name].map(lambda x: mapping.get(x, "Other"))
output_path = "D:/study/output/category.csv"
pdf.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"Đã xuất file phân loại tại: {output_path}")

xử lý batch 1 (50 items)...
xử lý batch 2 (50 items)...
xử lý batch 3 (50 items)...
xử lý batch 4 (50 items)...
xử lý batch 5 (50 items)...
xử lý batch 6 (50 items)...
xử lý batch 7 (50 items)...
xử lý batch 8 (50 items)...
xử lý batch 9 (50 items)...
xử lý batch 10 (50 items)...
xử lý batch 11 (50 items)...
xử lý batch 12 (50 items)...
xử lý batch 13 (50 items)...
xử lý batch 14 (50 items)...
xử lý batch 15 (50 items)...
xử lý batch 16 (50 items)...
xử lý batch 17 (50 items)...
xử lý batch 18 (50 items)...
xử lý batch 19 (50 items)...
xử lý batch 20 (50 items)...
xử lý batch 21 (50 items)...
xử lý batch 22 (50 items)...
xử lý batch 23 (50 items)...
xử lý batch 24 (50 items)...
xử lý batch 25 (50 items)...
xử lý batch 26 (50 items)...
xử lý batch 27 (50 items)...
xử lý batch 28 (50 items)...
xử lý batch 29 (50 items)...
xử lý batch 30 (50 items)...
xử lý batch 31 (50 items)...
xử lý batch 32 (50 items)...
xử lý batch 33 (50 items)...
xử lý batch 34 (50 items)...
xử lý batch 35 (50 item

In [5]:
folder_path = 'D:/study/dataset/log_search'
df = ETL_all_day(folder_path)
data6 = most_search(df, month=6)
data7 = most_search(df, month=7)

In [14]:
mapping_df = spark.read.csv("D:/study/output/category.csv", header= True, inferSchema=True)

data6 = (
    data6.join(mapping_df, on="Most_Search", how="inner")
         .select("user_id", "Most_Search", "Category")
         .withColumnRenamed("Most_Search", "Most_Search_T6")
         .withColumnRenamed("Category", "Category_T6")
)

data7 = (
    data7.join(mapping_df, on="Most_Search", how="inner")
         .select("user_id", "Most_Search", "Category")
         .withColumnRenamed("Most_Search", "Most_Search_T7")
         .withColumnRenamed("Category", "Category_T7")
)

In [18]:
df_all = data6.join(data7, on="user_id", how="inner")
condition = col("Category_T6") == col("Category_T7")
df_all = df_all.withColumn(
    "Category_change",
    when(condition, lit("NoChange"))
    .otherwise(concat(col("Category_T6"), lit(" - "), col("Category_T7")))
)


In [25]:
df_all.show()

+--------+--------------------+-----------+--------------------+-----------+------------------+
| user_id|      Most_Search_T6|Category_T6|      Most_Search_T7|Category_T7|   Category_change|
+--------+--------------------+-----------+--------------------+-----------+------------------+
| 0017684|pháp y tần minh: ...|    Mystery|       bác sĩ yo han|      Drama|   Mystery - Drama|
| 0019920|           thiếu nhi|  Animation|              bolero|      Music| Animation - Music|
| 0099596|          cuộc chiến|     Action|     hơn cả tình bạn|    Romance|  Action - Romance|
| 0153643|yêu em từ cái nhì...|    Romance|10 năm 3 tháng 30...|      Other|   Romance - Other|
|  016508|            Bigfoot |      Other|    taxi, em tên gì?|      Other|          NoChange|
| 0166772|          khủng long|  Animation|           siêu nhân|     Action|Animation - Action|
| 0177631|            trữ tình|      Music|            trữ tình|      Music|          NoChange|
| 0183627|         ngự giao ký|    Fanta

In [26]:
import_to_postgresql(df_all)

Data import successfully!
